In [152]:
import pandas as pd
import json

In [ ]:
with open("../data/mapping/attration.json", "r") as file:
    mapping = json.load(file)

df = pd.read_excel("../data/attractions/Location-Import.xlsx", sheet_name="Landmark")
df.head(5)

In [156]:
def process_dataframe_based_on_mapping(df, mapping):
    output = []

    for _, row in df.iterrows():
        record = {}
        for key, value in mapping.items():
            if isinstance(value, dict) and "column" in value:
                column_name = value["column"]
                if column_name in df.columns:
                    if "documentField" in value:
                        doc_field = value["documentField"]
                        if isinstance(row[column_name], pd.Timestamp):
                            # Special handling for date columns to represent in MongoDB Extended JSON format
                            date_str = row[column_name].strftime('%Y-%m-%dT%H:%M:%S')
                            record[doc_field] = {"$date": date_str}
                        else:
                            record[doc_field] = row[column_name]
        output.append(record)

    return output

result = process_dataframe_based_on_mapping(df, mapping)
print(json.dumps(result[:5], indent=4)) 

[
    {
        "id": "6455bad54673c82a34f4eccc",
        "landmark.lpNumber": "LP-00160",
        "landmark.landmarkType": "Individual Landmark",
        "landmark.designationDate": {
            "$date": "1968-11-12T00:00:00"
        }
    },
    {
        "id": "6470e61c9e866c1d68eb587f",
        "landmark.lpNumber": "LP-00161",
        "landmark.landmarkType": "Individual Landmark",
        "landmark.designationDate": {
            "$date": "1968-03-19T00:00:00"
        }
    },
    {
        "id": "64583f814673c82a34f4ecf6",
        "landmark.lpNumber": "LP-00162",
        "landmark.landmarkType": "Individual Landmark",
        "landmark.designationDate": {
            "$date": "1969-11-19T00:00:00"
        }
    },
    {
        "id": "6470d6d49e866c1d68eb587c",
        "landmark.lpNumber": "LP-00163",
        "landmark.landmarkType": "Individual Landmark",
        "landmark.designationDate": {
            "$date": "2016-08-10T00:00:00"
        }
    },
    {
        "id": "64582